# Web Scraping

## Importation des librairies

In [ ]:
# a mettre au propre une fois le projet fini, pour l'instant vide
# vcar selon le besoin au moment venue:
# voir si natif d'Anaconda sinon l'importer par la cmd

## Creation d'une methode generale

In [1]:
from playhouse.sqlite_ext import SqliteExtDatabase

# à importer par la cmd
from peewee import *

In [2]:
# creation de la base sql
db = SqliteExtDatabase('database.sqlite')

In [3]:
# creation de classe: nouveau type d'objet
class Annonce(Model):
    # id = "pap-123456789"
    id = CharField(unique=True, primary_key=True)
    # site = [pap, lbc, logic-immo, seloger]
    site = CharField()
    created = DateTimeField()
    title = CharField()
    description = TextField(null=True)
    telephone = TextField(null=True)
    price = FloatField()
    charges = FloatField(null=True)
    surface = FloatField()
    rooms = IntegerField()
    bedrooms = IntegerField(null=True)
    city = CharField()
    link = CharField()
    picture = CharField(null=True)
    posted2trello = BooleanField(default=False)

    class Meta:
        database = db
        order_by = ('-created',)


def create_tables():
    with db:
        db.create_tables([Annonce])

## Scrapping des différents sites: Seloger.com, LeBonCoin.fr, Biend'Ici

### Scraping SeLoger.com

In [4]:
import requests

import xml.etree.ElementTree as ET

from datetime import datetime

In [8]:
#module qui récupère les annonces de SeLoger.com

def search(parameters):
    # Préparation des paramètres de la requête
    payload = {
        'px_loyermin': parameters['price'][0],
        'px_loyermax': parameters['price'][1],
        'surfacemin': parameters['surface'][0],
        'surfacemax': parameters['surface'][1],
        # Si parameters['rooms'] = (2, 4) => "2,3,4"
        'nbpieces': list(range(parameters['rooms'][0], parameters['rooms'][1] + 1)),
        # Si parameters['bedrooms'] = (2, 4) => "2,3,4"
        'nb_chambres': list(range(parameters['bedrooms'][0], parameters['bedrooms'][1] + 1)),
        'ci': [int(cp[2]) for cp in parameters['cities']]
    }
    
     # Insertion des paramètres propres à LeBonCoin
    payload.update(parameters['seloger'])

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.167 Safari/537.36'}

    request = requests.get("https://www.seloger.com/list.htm?types=1&projects=2,5&enterprise=0&natures=1,2,4&places=[{ci:750056}]&qsVersion=1.0", params=payload, headers=headers)
    
    xml_root = ET.fromstring(request.text)
    
    for annonceNode in xml_root.findall('annonces/annonce'):
        # Seconde requête pour obtenir la description de l'annonce
        _payload = {'noAudiotel': 1, 'idAnnonce': annonceNode.findtext('idAnnonce')}
        _request = requests.get("http://ws.seloger.com/annonceDetail_4.0.xml", params=_payload, headers=headers)
        
        photos = list()
        for photo in annonceNode.find("photos"):
            photos.append(photo.findtext("stdUrl"))

        annonce, created = Annonce.create_or_get(
            id='seloger-' + annonceNode.find('idAnnonce').text,
            site='SeLoger',
            # SeLoger peut ne pas fournir de titre pour une annonce T_T
            title="Appartement " + annonceNode.findtext('nbPiece') + " pièces" if annonceNode.findtext('titre') is None else annonceNode.findtext('titre'),
            description=ET.fromstring(_request.text).findtext("descriptif"),
            telephone=ET.fromstring(_request.text).findtext("contact/telephone"),
            created=datetime.strptime(annonceNode.findtext('dtCreation'), '%Y-%m-%dT%H:%M:%S'),
            price=annonceNode.find('prix').text,
            charges=annonceNode.find('charges').text,
            surface=annonceNode.find('surface').text,
            rooms=annonceNode.find('nbPiece').text,
            bedrooms=annonceNode.find('nbChambre').text,
            city=annonceNode.findtext('ville'),
            link=annonceNode.findtext('permaLien'),
            picture=photos
        )

        if created:
            annonce.save() 

### Scraping LeBonCoin.fr

In [10]:
# Virginie

### Scraping BienDici.com

In [11]:
# Virginie et moi à faire

## Programme principal: pour l'instant fait que sur seloger

In [12]:
import os
import sys
import json

In [10]:
import logging

In [23]:
logging.basicConfig(level=logging.INFO)

In [24]:
os.chdir(os.path.dirname(sys.argv[0]))

In [27]:
create_tables()

In [29]:
print(db)

In [1]:
# Chargement des paramètres de recherche depuis le fichier JSON
with open("C:/Users/asial/projet_ws/parameters.json", encoding='utf-8') as parameters_data:
    parameters = json.load(parameters_data)

In [11]:
# Recherche et insertion en base
if "seloger" in parameters['ad-providers']:
    logging.info("Retrieving from seloger")
    search(parameters)

ValueError: invalid literal for int() with base 10: 'n'

In [2]:
test=parameters.get('cities')

In [3]:
print(test)

['Nanterre', 92000, 920050]


In [4]:
type(test[1])

int

In [5]:
for cp in parameters['cities']:
    print(cp)

Nanterre
92000
920050


In [6]:
print(test[1])

92000


Pourquoi dans la fonction search 'ci' récupère les éléments de Nanterre et non de la liste [] donc 92000 ??? T_T